In [2]:
import os
import glob

os.environ["CONFIG_APP_DIR"] = "../tests/test_custom/config_app/"

In [3]:
import sys
import pickle
from omegaconf import OmegaConf
from etl_pipeline.config import pipeline_config
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine

In [4]:
from etl_pipeline.config import columns_namespace as cn

In [5]:
ETL_SAMPLES_PATH = '/v/region/na/appl/itlnc/s8/data/uat/silent8/name-screening/etl_samples/complete_info/'

In [6]:
DATASET_NAME = 'WM' # 'ISG_Daily' # 'WM' # 'ISG_Weekly 
DATASET_TYPE =  'Address' # 'Address' # Party # Account
DATASET = DATASET_NAME + '_' + DATASET_TYPE

In [7]:
complete_info = glob.glob(ETL_SAMPLES_PATH + DATASET + '/*.json')

### Testing flow

In [8]:
from pipelines.ms.ms_pipeline import MSPipeline

engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

In [9]:
with open(f'../notebooks/sample/wm_party_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)


payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"][cn.MATCH_RECORDS]))]


In [10]:
payload = pipeline.transform_standardized_to_cleansed(payload_json)
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [11]:
import pandas as pd

In [12]:
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

In [13]:
out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_party_types_aggregated,wl_all_party_types_aggregated,ap_all_pobs_aggregated,wl_all_pobs_aggregated,ap_all_residencies_aggregated,wl_all_residencies_aggregated,ap_all_santioned_countries_aggregated,wl_all_santioned_countries_aggregated
0,"[02/31/1900, 04/31/1910]","[MAY 6, 1981, //]","[154421273, 022368917]",[],[],[Joe Ding],"[Pladimir Vutin, Eva Pladimirova]",[Joe Ding],"[Individual, Individual]",[],[],[Joe Ding],"[United States, United States]",[],[],[Joe Ding]
1,"[02/31/1900, 04/31/1910]","[MAY 6, 1981, //]","[154421273, 022368917]",[],[],[Joe Dong],"[Pladimir Vutin, Eva Pladimirova]",[Joe Dong],"[Individual, Individual]",[],[],"[PL, Poland, Joe Dong]","[United States, United States]","[PL, Poland]",[],"[PL, Poland, Joe Dong]"


# check all test jsons

In [14]:
source_files = ['../notebooks/sample/wm_party_in_payload_format.json', '../notebooks/sample/wm_address_in_payload_format.json']

In [15]:
from tqdm import tqdm
from dataclasses import asdict


def dump_application(source_files, target_path="notebooks/application"):
    os.makedirs(target_path, exist_ok=True)
    for info in tqdm(source_files):
        with open(info, 'r') as file:
            payload = json.load(file)

        payload_json = {key: payload[key] for key in sorted(payload)}
        payload_json = PayloadLoader().load_payload_from_json(payload_json)
        payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"][cn.MATCH_RECORDS]))]
        payload = pipeline.transform_standardized_to_cleansed(payload_json)
        new_payloads = pipeline.transform_cleansed_to_application(payload)

        for i in new_payloads:
            for m in i['alertedParty']['inputRecordHist']['inputRecords']:
                for key in m['INPUT_FIELD']:
                    m['INPUT_FIELD'][key] = asdict(m['INPUT_FIELD'][key])

        with open(os.path.join(target_path, "application_" + os.path.basename(info)), 'w') as file:
            json.dump(new_payloads, file)

    
            
    

In [16]:
dump_application(source_files)

100%|██████████| 2/2 [00:00<00:00, 68.14it/s]


In [18]:
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]